In [39]:
import os
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [40]:
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [41]:
data = sparkSession.read.parquet("/data/sample264")
dataforNext=sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")
dataforNext=dataforNext.select('userId',  \
                         dataforNext.trackId.alias('trackId1'), \
                         dataforNext.timestamp.alias('timestamp1'))

In [42]:
def normalization(data, key1, key2, field, threshold): 
    
    window = Window.partitionBy(key1).orderBy(col(field).desc())
        
    top = data.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= threshold) \
        .drop(col("row_number")) 
        
    df = top.groupBy(col(key1)).agg(col(key1), sum(col(field)).alias("sum_" + field))
   
    normalizedData = top.join(df, key1, "inner") \
        .withColumn("norm_" + field, col(field) / col("sum_" + field)) \
        .cache()

    return normalizedData

In [43]:
dataJoined = data.join(dataforNext, data.userId == dataforNext.userId )\
                       .filter(data.trackId!=dataforNext.trackId1) \
                       .filter((dataforNext.timestamp1-data.timestamp<=420) & (dataforNext.timestamp1-data.timestamp>0) ) \
                    .select(data.userId,  \
                         data.trackId.alias('trackId1'), \
                         dataforNext.trackId1.alias('trackId2')) \
                .withColumn("id", when((col('trackId2')>=col('trackId1')),col('trackId1')).otherwise(col('trackId2'))) \
                .withColumn("id2", when((col('trackId1')<=col('trackId2')),col('trackId2')).otherwise(col('trackId1'))) 

tracks = dataJoined.groupBy('id', 'id2').count()
tracksNorm = normalization(tracks, "id","id2", "count", 40)
window = Window.orderBy(col("norm_count").desc())
trackList = tracksNorm.withColumn("position", rank().over(window)) \
                         .filter(col("position") <= 40)\
                         .sort(["norm_count","id","id2"],ascending=[0,1,1])\
                        .select(col("id"), col("id2"))\
                          .take(40)

In [44]:
for val in trackList:
    print "%s %s" % val

798256 923706
798319 837992
798322 876562
798331 827364
798335 840741
798374 816874
798375 810685
798379 812055
798380 840113
798396 817687
798398 926302
798405 867217
798443 905923
798457 918918
798460 891840
798461 940379
798470 840814
798474 963162
798477 883244
798485 955521
798505 905671
798545 949238
798550 936295
798626 845438
798691 818279
798692 898823
798702 811440
798704 937570
798725 933147
798738 894170
798745 799665
798782 956938
798801 950802
798820 890393
798833 916319
798865 962662
798931 893574
798946 946408
799012 809997
799024 935246
